In [1]:
!pip install pdfplumber PyGithub transformers sentence-transformers faiss-cpu

import os, re, pdfplumber
import pandas as pd
from github import Github
from sentence_transformers import SentenceTransformer, util
from transformers import pipeline
import spacy
from IPython.display import display
from google.colab import files


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 53.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 53.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 65.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 72.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 41.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.

In [19]:
nlp = spacy.load("en_core_web_sm")
encoder = SentenceTransformer("all-MiniLM-L6-v2")
rag = pipeline("question-answering", model="facebook/rag-token-base")
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")


Some weights of the model checkpoint at facebook/rag-token-base were not used when initializing RagTokenForGeneration: ['rag.question_encoder.question_encoder.bert_model.pooler.dense.bias', 'rag.question_encoder.question_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing RagTokenForGeneration from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RagTokenForGeneration from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called 

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


In [3]:
uploaded = files.upload()
resume_files = list(uploaded.keys())

Saving Tejas_Venugopalan.pdf to Tejas_Venugopalan.pdf


In [14]:
def extract_resume_text(path):
    text = ""
    with pdfplumber.open(path) as pdf:
        for page in pdf.pages:
            text += page.extract_text() or ''
    return text
def extract_fields(text):
    name = None
    github = None
    skills = []
    projects = []
    cgpa = None
    has_leadership = False
    experience = []

    doc = nlp(text)

    # Name
    for ent in doc.ents:
        if ent.label_ == "PERSON":
            name = ent.text
            break

    # GitHub
    github_match = re.search(r"(https:\/\/github\.com\/[^\s\)]+)", text)
    github = github_match.group(1) if github_match else None

    # CGPA
    cgpa_match = re.search(r"(?:CGPA|GPA)[^\d]*(\d\.\d+)", text, re.IGNORECASE)
    cgpa = float(cgpa_match.group(1)) if cgpa_match else None

    # Skills
    for line in text.split("\n"):
        if "skills" in line.lower():
            skills += re.findall(r"\b[A-Za-z\+#]+", line)

    # Projects
    for line in text.split("\n"):
        if "project" in line.lower():
            projects.append(line.strip())

    # Leadership
    keywords = ["president", "secretary", "lead", "organizer", "founder", "coordinator", "captain"]
    if any(k.lower() in text.lower() for k in keywords):
        has_leadership = True

    # Experience (basic heuristic: lines with work/intern/lead/etc.)
    exp_keywords = ["worked", "intern", "experience", "lead", "developed", "engineer", "manager", "analyst"]
    for line in text.split('\n'):
        if any(k in line.lower() for k in exp_keywords):
            experience.append(line.strip())

    return {
        "name": name,
        "github": github,
        "cgpa": cgpa,
        "skills": list(set(skills)),
        "projects": projects,
        "has_leadership": has_leadership,
        "experience": experience,
        "raw_text": text
    }


In [5]:
def fetch_github_readmes(github_url, token=""):
    try:
        username = github_url.split("/")[-1]
        g = Github(token)
        user = g.get_user(username)
    except:
        return []

    repos = []
    for repo in user.get_repos():
        try:
            readme = repo.get_readme()
            content = readme.decoded_content.decode()
            repos.append(content)
        except:
            continue
    return repos


In [6]:
def chunk_text(text, chunk_size=500):
    return [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]

def rag_verify(claim, repo_chunks):
    context = " ".join(repo_chunks)[:2000]
    try:
        result = rag(question=claim, context=context)
        return result['answer'], result['score']
    except:
        return "Could not process", 0.0

def verify_claims_rag(resume_projects, github_readmes):
    results = []
    repo_chunks = []
    for readme in github_readmes:
        repo_chunks.extend(chunk_text(readme))

    for claim in resume_projects:
        query = f"Does this GitHub repository support this claim: '{claim}'?"
        answer, score = rag_verify(query, repo_chunks)
        results.append({
            "claim": claim,
            "answer": answer,
            "verified": "yes" in answer.lower() or score > 0.4,
            "confidence": round(score, 3),
        })
    return results


In [8]:
def generate_report(parsed, scores, final_score):
    summary = f"""
    Candidate: {parsed['name']}
    🔹 Final Score: {final_score} / 100
    📁 Project Claim Verification: {round(scores['claim_verification'] * 10, 2)} / 10
    📂 Project–JD Similarity: {round(scores['project_jd'] * 10, 2)} / 10
    🎓 CGPA Contribution: {round(scores['cgpa'] * 20, 2)} / 20
    🧠 Skills Match: {round(scores['skills_match'] * 20, 2)} / 20
    👑 Leadership Bonus: {20 if scores['leadership'] else 0} / 20
    🎁 Static Bonus: 20 / 20
    """
    return summary

In [23]:
def generate_summary(parsed, job_description, summarizer):
    resume_text = parsed["raw_text"]

    # --- Part 1: Resume Summary ---
    input_text = f"Summarize the candidate's resume:\n{resume_text}"
    resume_summary = summarizer(input_text[:1024], max_length=100, min_length=40, do_sample=False)[0]['summary_text']

    # --- Part 2: JD Alignment ---
    jd_alignment = []

    if parsed.get("cgpa") and parsed["cgpa"] >= 7.5:
        jd_alignment.append(f"CGPA of {parsed['cgpa']} meets the requirement.")
    else:
        jd_alignment.append(f"CGPA of {parsed.get('cgpa', 'N/A')} is below the preferred threshold.")

    skills = set(s.lower() for s in parsed.get("skills", []))
    if "python" in skills:
        jd_alignment.append("Strong Python skills identified.")
    if "fastapi" in skills:
        jd_alignment.append("FastAPI experience confirmed.")
    if "mongodb" in skills:
        jd_alignment.append("MongoDB experience confirmed.")
    if parsed.get("has_leadership"):
        jd_alignment.append("Leadership experience demonstrated.")

    # Project-based match (basic scoring logic)
    matches = [p for p in parsed.get("projects", []) if any(k in p.lower() for k in ["api", "real-time", "backend", "fastapi", "ml", "transformer"])]
    if matches:
        jd_alignment.append(f"{len(matches)} project(s) align with real-time systems or backend/API development.")

    alignment_summary = " ".join(jd_alignment)

    # --- Final Combined Summary ---
    return f"🧾 **Resume Summary:** {resume_summary}\n\n📌 **JD Alignment:** {alignment_summary}"


In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# ---- Score: How closely resume projects match JD ----
def score_projects_to_jd(projects, jd):
    if not projects:
        return 0.0
    vectorizer = TfidfVectorizer().fit_transform([jd] + projects)
    vectors = vectorizer.toarray()
    jd_vec = vectors[0]
    project_vecs = vectors[1:]
    sims = cosine_similarity([jd_vec], project_vecs)[0]
    return float(np.mean(sims))

# ---- Score: Skill–JD overlap ----
def score_skills_match(skills, jd):
    jd_words = set(jd.lower().split())
    matches = [s for s in skills if s.lower() in jd_words]
    return len(matches) / max(1, len(skills))

# ---- Score: Leadership detection ----
def score_leadership(experience):
    if not experience:
        return 0.0
    keywords = ["lead", "captain", "head", "president", "organized", "coordinator", "mentor"]
    for line in experience:
        if any(k in line.lower() for k in keywords):
            return 1.0
    return 0.0


In [12]:
def compute_score(parsed, jd, verified_claims):
    scores = {
        "claim_verification": sum([v['confidence'] for v in verified_claims]) / max(1, len(verified_claims)),
        "project_jd": score_projects_to_jd(parsed["projects"], jd),
        "cgpa": parsed["cgpa"] / 10 if parsed["cgpa"] else 0.0,
        "skills_match": score_skills_match(parsed["skills"], jd),
        "leadership": score_leadership(parsed["experience"]),
        "bonus": 1.0  # always awarded
    }

    weights = {
        "claim_verification": 0.10,
        "project_jd": 0.10,
        "cgpa": 0.20,
        "skills_match": 0.20,
        "leadership": 0.20,
        "bonus": 0.20
    }

    final = sum([scores[k] * weights[k] for k in scores])
    return round(final * 100, 2), scores


In [25]:
# =============================
# 🚀 Run Full Pipeline with Detailed JD
# =============================
job_description = """
We are hiring Software Development Engineers who are passionate about building scalable backend systems,
designing APIs, and working with real-time data. Candidates must have strong fundamentals in Python
and experience with FastAPI, MongoDB, and RESTful services.

We value open-source contributions and expect candidates to showcase real-world projects, preferably on GitHub,
demonstrating skills in backend development, cloud-native systems, or machine learning pipelines.

The ideal candidate will have:
- Hands-on experience with FastAPI or Flask/Django
- Worked with NoSQL databases like MongoDB
- Projects involving real-time data processing or API integrations
- Basic ML pipeline knowledge (scikit-learn, transformers, etc.)
- Demonstrated leadership in technical clubs, events, or open-source communities
- Good academic record with a CGPA above 7.5
"""

results = []

for file in resume_files:
    print(f"\n🔍 Processing: {file}")
    text = extract_resume_text(file)
    parsed = extract_fields(text)
    github_readmes = fetch_github_readmes(parsed["github"]) if parsed["github"] else []
    verified_claims = verify_claims_rag(parsed["projects"], github_readmes)
    final_score, breakdown = compute_score(parsed, job_description, verified_claims)
    report = generate_report(parsed, breakdown, final_score)
    summary = generate_summary(parsed, job_description, summarizer)

    results.append({
        "name": parsed["name"],
        "github": parsed["github"],
        "cgpa": parsed["cgpa"],
        "score": final_score,
        "report": report,
        "summary": summary,
        **breakdown
    })

df = pd.DataFrame(results).sort_values(by="score", ascending=False)
display(df[["name", "score", "cgpa", "report"]])



🔍 Processing: Tejas_Venugopalan.pdf


,name,score,cgpa,report
0,CNR Rao Scholarship,67.2,0.874,\n Candidate: CNR Rao Scholarship\n 🔹 Fi...


In [26]:
df.to_csv("grifter_final_ranking.csv", index=False)
files.download("grifter_final_ranking.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>